# Limpieza de datos.

Para la fase 1 del proyecto sustituto del curso Modelos y Simulaciones I, 202402, Universidad de Antioquia, se ha tomado como objeto de estudio la competencia de Kaggle [New York City Taxi Trip Duration](https://www.kaggle.com/competitions/nyc-taxi-trip-duration/), utilizando como guía el notebook [Beginner Friendly Approach](https://www.kaggle.com/code/jaysabnis/new-york-trip-duration-beginner-friendly-approach/notebook) del usuario Jay Sabnis.

En este notebook se extrajo y se reordenó el código correspondiente a limpieza de datos, el cual está compuesto principalmente de 2 funciones, **haversine_distance**, la cual hace una transformación de coordenadas cartesianas a coordenadas polares para hacer un cálculo aproximado de la distancia recorrida en cada viaje. La función **clean_data**, se encarga de orquestar las transformaciones necesarias sobre los datos para que estos puedan ser considerados aptos para ser procesados por los algoritmos de entrenamiento de los diferentes modelos dé IA.

In [1]:
# Se clona el repositorio de github en el entrno del notebook.
# Se descomprimer el zip que contiene los datos a limpiar.

!git clone https://github.com/franco-arroyave/Modelos-y-Simulaciones-I
!unzip -uq "/content/Modelos-y-Simulaciones-I/data/train.zip" -d "/content/"

Cloning into 'Modelos-y-Simulaciones-I'...
unzip:  cannot find or open /content/Modelos-y-Simulaciones-I/data/train.zip, /content/Modelos-y-Simulaciones-I/data/train.zip.zip or /content/Modelos-y-Simulaciones-I/data/train.zip.ZIP.


In [2]:
#Cargar librerias base.

import math
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Función utilizada para calcular la distancia de la carrera.

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points
    on the Earth's surface using the Haversine formula.

    Parameters:
        lat1, lon1 (float): Latitude and longitude of the first point (in degrees).
        lat2, lon2 (float): Latitude and longitude of the second point (in degrees).

    Returns:
        float: Distance between the two points in kilometers.
    """
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    radius_of_earth = 6371  # Radius of the Earth in kilometers
    distance = radius_of_earth * c

    return distance

In [4]:
# Función encagada de orquestar la limpieza de los datos.

def clean_data(df):
  """
  Limpia los datos de entrada, aplicando cambios de formato, eliminación de columnas y manejo de outliers.

  Parameters
  ----------
  df : dataframe pandas

  Returns
  -------
  train_data : dataframe pandas
  """

  day_map={
    'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,
    'Friday':4,'Saturday':5,'Sunday':6,
  }

  cp_df=df.copy()
  cp_df["pickup_datetime"] = pd.to_datetime(cp_df["pickup_datetime"])
  cp_df["pickup_day"] = cp_df["pickup_datetime"].dt.day_name()
  cp_df["pickup_time"]=cp_df["pickup_datetime"].dt.hour
  cp_df['distance_km'] = cp_df.apply(lambda row: haversine_distance(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
  cp_df.drop(["pickup_datetime","dropoff_datetime"],axis=1,inplace=True)
  cp_df["store_and_fwd_flag"]=cp_df.store_and_fwd_flag.map({"N":0,"Y":1})

  outliers_in_trip_duration_in_seconds=cp_df[(cp_df["trip_duration"]>8000)&(cp_df["distance_km"]<25)]
  cp_df=cp_df.drop(outliers_in_trip_duration_in_seconds.index, axis=0)
  outliers_in_trip_duration_in_seconds_2=cp_df[(cp_df["trip_duration"]>30000)&(cp_df["distance_km"]<200)]
  cp_df= cp_df.drop(outliers_in_trip_duration_in_seconds_2.index, axis=0)
  outliers_in_trip_duration_in_seconds_3=cp_df[(cp_df["trip_duration"]<50000)&(cp_df["distance_km"]>200)]
  cp_df= cp_df.drop(outliers_in_trip_duration_in_seconds_3.index, axis=0)
  outliers_in_trip_duration_in_seconds_4=cp_df[(cp_df["trip_duration"]>0)&(cp_df["distance_km"]==0)]
  cp_df= cp_df.drop(outliers_in_trip_duration_in_seconds_4.index, axis=0)

  cp_df['passenger_count'] = pd.Categorical(cp_df['passenger_count'])
  outliers_in_trip_duration_in_seconds_5=cp_df[(cp_df["passenger_count"]==0)&(cp_df["trip_duration"]>1500)]
  cp_df= cp_df.drop(outliers_in_trip_duration_in_seconds_5.index, axis=0)
  cp_df['ordinal_pick_up_day']=cp_df["pickup_day"].map(day_map)
  train_data=cp_df.drop(["id","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","pickup_day"],axis=1)

  train_data['passenger_count'] = train_data['passenger_count'].astype(int)

  return train_data

In [5]:
#Carga los datos que van a ser limpiados.

taxi_data_train=pd.read_csv("train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
#Se ejecutado la función encargada de la limpieza sobre los datos correspondientes.

data_train = clean_data(taxi_data_train)

In [ ]:
#Un vistados a los datos luego de ser preprocesados.

data_train.describe()

In [ ]:
data_train.info()

In [ ]:
#Datos exportados a un archivo CSV para ser procesados en otra instancia.

data_train.to_csv("data_clean.csv",index=False)